In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from os import makedirs, listdir
from os.path import join, isfile, isdir, exists, splitext
from pprint import pprint
import json
Settings = json.load(open('settings.txt'))
pprint(Settings)
print("")
from pak.datasets.MOT import MOT16
from pak import utils
import sys
sys.path.append('../')

from cabbage.features.deepmatching import DeepMatching
import cabbage.features.spatio as st

# -------------------------

root = Settings['data_root']
mot16 = MOT16(root)

delta_max = 5
dm = DeepMatching(Settings['deepmatch'], join(root, 'deep_matching'),
                 delta_max=delta_max)

VIDEO = "MOT16-11"
img_loc = mot16.get_test_imgfolder(VIDEO)

X, Y_det, Y_gt = mot16.get_train(VIDEO, memmapped=True)

{'data_root': '/Users/basheq/Desktop/RGBD_Lab/Data',
 'deepmatch': '/Users/basheq/Desktop/RGBD_Lab/deepmatching_1.2.2_c++/deepmatching'}

/Users/basheq/Desktop/RGBD_Lab/Data/MOT16 found :)
MOT16: load memmapped file /Users/basheq/Desktop/RGBD_Lab/Data/MOT16/train/MOT16-11/data.memmap
MOT16 X loaded
MOT16 Y_det loaded
MOT16 Y_gt loaded


## Generate Deep Matches

For performance reasons we need to pre-generate the deep matches using the API functions

In [2]:
dm.generate_matches(img_loc, VIDEO)

{DM}: solve 573 -> 573
{DM}: solve 573 -> 574
{DM}: solve 573 -> 575
{DM}: solve 573 -> 576
{DM}: solve 573 -> 577
{DM}: solve 573 -> 578
{DM}: solve 574 -> 574
{DM}: solve 574 -> 575
{DM}: solve 574 -> 576
{DM}: solve 574 -> 577
{DM}: solve 574 -> 578
{DM}: solve 574 -> 579
{DM}: solve 575 -> 575
{DM}: solve 575 -> 576
{DM}: solve 575 -> 577
{DM}: solve 575 -> 578
{DM}: solve 575 -> 579
{DM}: solve 575 -> 580
{DM}: solve 576 -> 576
{DM}: solve 576 -> 577
{DM}: solve 576 -> 578
{DM}: solve 576 -> 579
{DM}: solve 576 -> 580
{DM}: solve 576 -> 581
{DM}: solve 577 -> 577
{DM}: solve 577 -> 578
{DM}: solve 577 -> 579
{DM}: solve 577 -> 580
{DM}: solve 577 -> 581
{DM}: solve 577 -> 582
{DM}: solve 578 -> 578
{DM}: solve 578 -> 579
{DM}: solve 578 -> 580
{DM}: solve 578 -> 581
{DM}: solve 578 -> 582
{DM}: solve 578 -> 583
{DM}: solve 579 -> 579
{DM}: solve 579 -> 580
{DM}: solve 579 -> 581
{DM}: solve 579 -> 582
{DM}: solve 579 -> 583
{DM}: solve 579 -> 584
{DM}: solve 580 -> 580
{DM}: solve

{DM}: solve 632 -> 635
{DM}: solve 632 -> 636
{DM}: solve 632 -> 637
{DM}: solve 633 -> 633
{DM}: solve 633 -> 634
{DM}: solve 633 -> 635
{DM}: solve 633 -> 636
{DM}: solve 633 -> 637
{DM}: solve 633 -> 638
{DM}: solve 634 -> 634
{DM}: solve 634 -> 635
{DM}: solve 634 -> 636
{DM}: solve 634 -> 637
{DM}: solve 634 -> 638
{DM}: solve 634 -> 639
{DM}: solve 635 -> 635
{DM}: solve 635 -> 636
{DM}: solve 635 -> 637
{DM}: solve 635 -> 638
{DM}: solve 635 -> 639
{DM}: solve 635 -> 640
{DM}: solve 636 -> 636
{DM}: solve 636 -> 637
{DM}: solve 636 -> 638
{DM}: solve 636 -> 639
{DM}: solve 636 -> 640
{DM}: solve 636 -> 641
{DM}: solve 637 -> 637
{DM}: solve 637 -> 638
{DM}: solve 637 -> 639
{DM}: solve 637 -> 640
{DM}: solve 637 -> 641
{DM}: solve 637 -> 642
{DM}: solve 638 -> 638
{DM}: solve 638 -> 639
{DM}: solve 638 -> 640
{DM}: solve 638 -> 641
{DM}: solve 638 -> 642
{DM}: solve 638 -> 643
{DM}: solve 639 -> 639
{DM}: solve 639 -> 640
{DM}: solve 639 -> 641
{DM}: solve 639 -> 642
{DM}: solve

KeyboardInterrupt: 

## Get a Match

In [ ]:
M = dm.get_match(VIDEO, frame1=1, frame2=2)
print(M.shape)

In [ ]:
frame1, _, x, y, w, h, _, _, _ = Y_gt[0]


fig = plt.figure(figsize=(16,8))
ax = fig.add_subplot(121)
ax.imshow(X[0])
bbX, bbY = utils.bb_to_plt_plot(x, y, w, h)
ax.plot(bbX, bbY, linewidth=3, c='red')
bb1 = (x,y,w,h)

frame2, _, x, y, w, h, _, _, _ = Y_gt[2]
ax = fig.add_subplot(122)
ax.imshow(X[1])
bbX, bbY = utils.bb_to_plt_plot(x, y, w, h)
ax.plot(bbX, bbY, linewidth=3, c='red')
bb2 = (x,y,w,h)


frame3, _, x, y, w, h, _, _, _ = Y_gt[601]
bbX, bbY = utils.bb_to_plt_plot(x, y, w, h)
ax.plot(bbX, bbY, linewidth=3, c='blue')
bb3 = (x,y,w,h)
# ----------------------

cost1 = dm.calculate_cost(VIDEO, frame1, bb1, frame2, bb2)
cost2 = dm.calculate_cost(VIDEO, frame1, bb1, frame3, bb3)
print("dm cost (red vs red):\t", cost1)
print("dm cost (red vs blue):\t", cost2)

cost1 = st.calculate(bb1, bb2)
cost2 = st.calculate(bb1, bb3)
print("st cost (red vs red):\t", cost1)
print("st cost (red vs blue):\t", cost2)
print(X.shape)

plt.show()